In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error

import tensorflow as tf

In [2]:
os.getcwd()

'D:\\mywork\\DACON_CONTEST_2019\\SUHYUN'

In [32]:
os.chdir('D:\\mywork\\DACON_CONTEST_2019')

In [88]:
cluster = pd.read_csv('./SOOMIN/presentation/clusterIDs_k2.csv').T
cluster.columns = ['TEST_Cluster1', 'TEST_Cluster2', 'TRAIN_Cluster1', 'TRAIN_Cluster2']
cluster = cluster.iloc[1:]
cluster

,TEST_Cluster1,TEST_Cluster2,TRAIN_Cluster1,TRAIN_Cluster2
0,X10,X105,X11,X100
1,X102,X106,X113,X1000
2,X16,X107,X119,X1001
3,X18,X110,X12,X1002
4,X26,X111,X15,X1003
5,X30,X116,X2,X1004
6,X37,X120,X23,X1005
7,X41,X123,X27,X1006
8,X44,X124,X28,X1007
9,X5,X125,X40,X1008


In [89]:
raw_TRAIN = pd.read_csv('./DACON_DATASET/train.csv')
raw_TRAIN['Time'] = pd.to_datetime(raw_TRAIN['Time']) 
raw_TRAIN = raw_TRAIN.set_index('Time')

raw_TEST = pd.read_csv('./DACON_DATASET/test.csv')
raw_TEST['Time'] = pd.to_datetime(raw_TEST['Time']) 
raw_TEST = raw_TEST.set_index('Time')

raw_TEST

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275,0.021,NaN
2017-07-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.222,0.021,NaN
2017-07-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237,0.021,NaN
2017-07-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.229,0.020,NaN
2017-07-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202,0.021,NaN
2017-07-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.251,0.021,NaN
2017-07-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.239,0.021,NaN
2017-07-01 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.207,0.020,NaN
2017-07-01 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.176,0.021,NaN


In [106]:
train_cluster = raw_TRAIN.T
train_cluster['Cluster'] = np.nan
train_cluster

Time,2016-07-26 11:00:00,2016-07-26 12:00:00,2016-07-26 13:00:00,2016-07-26 14:00:00,2016-07-26 15:00:00,2016-07-26 16:00:00,2016-07-26 17:00:00,2016-07-26 18:00:00,2016-07-26 19:00:00,2016-07-26 20:00:00,...,2018-06-30 15:00:00,2018-06-30 16:00:00,2018-06-30 17:00:00,2018-06-30 18:00:00,2018-06-30 19:00:00,2018-06-30 20:00:00,2018-06-30 21:00:00,2018-06-30 22:00:00,2018-06-30 23:00:00,Cluster
X692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.328,0.498,0.396,0.331,0.271,0.341,0.535,0.379,0.388,NaN
X1272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.484,0.543,0.631,0.666,0.754,0.827,0.516,0.494,0.482,NaN
X553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.454,0.468,0.397,0.415,0.327,0.478,0.558,0.605,0.651,NaN
X1299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.180,0.171,0.170,0.312,0.468,0.356,0.496,0.320,0.284,NaN
X4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.445,0.344,0.475,0.288,0.314,0.434,0.479,0.555,0.452,NaN
X1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.329,0.444,0.567,0.628,0.769,0.680,0.590,0.604,0.629,NaN
X1010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.172,0.388,0.371,1.255,0.790,0.819,0.910,1.044,1.116,NaN
X1216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.330,0.887,0.600,0.611,0.558,0.470,0.424,0.412,0.400,NaN
X1047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.153,0.155,0.162,0.165,0.149,0.160,0.468,0.480,0.274,NaN


In [111]:
Cluster_name = 'TRAIN_Cluster2'
# ['TRAIN_Cluster1', 'TRAIN_Cluster2']

for i in cluster[Cluster_name].dropna().tolist():
    train_cluster['Cluster'][i] = Cluster_name

train_cluster.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [133]:
test_cluster = raw_TEST.T
test_cluster['Cluster'] = np.nan
test_cluster

Time,2017-07-01 00:00:00,2017-07-01 01:00:00,2017-07-01 02:00:00,2017-07-01 03:00:00,2017-07-01 04:00:00,2017-07-01 05:00:00,2017-07-01 06:00:00,2017-07-01 07:00:00,2017-07-01 08:00:00,2017-07-01 09:00:00,...,2018-06-30 15:00:00,2018-06-30 16:00:00,2018-06-30 17:00:00,2018-06-30 18:00:00,2018-06-30 19:00:00,2018-06-30 20:00:00,2018-06-30 21:00:00,2018-06-30 22:00:00,2018-06-30 23:00:00,Cluster
X26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.179,1.703,1.504,1.271,1.255,1.596,1.379,1.000,0.976,NaN
X303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.211,0.168,0.140,0.495,0.175,1.692,0.883,0.539,0.481,NaN
X241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.245,0.211,0.215,0.372,0.430,0.332,0.435,0.268,0.220,NaN
X435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.208,0.207,0.199,0.331,0.429,0.366,0.497,0.459,0.418,NaN
X402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.188,0.179,0.213,0.232,0.169,0.159,0.166,0.463,0.646,NaN
X352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.757,0.523,0.530,0.625,0.739,0.255,0.233,0.454,0.419,NaN
X305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.344,0.316,0.513,0.561,0.573,0.501,0.634,0.676,0.680,NaN
X350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.345,0.324,0.322,0.398,0.341,0.281,0.401,0.543,0.499,NaN
X326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.272,0.571,0.148,0.134,0.127,0.126,0.439,0.737,0.304,NaN
X299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.110,0.301,0.223,0.347,0.286,0.230,0.346,0.466,0.405,NaN


In [136]:
Cluster_name = 'TEST_Cluster1'
# ['TEST_Cluster1', 'TEST_Cluster2']

for i in cluster[Cluster_name].dropna().tolist():
    test_cluster['Cluster'][i] = Cluster_name

test_cluster.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Time,2017-07-01 00:00:00,2017-07-01 01:00:00,2017-07-01 02:00:00,2017-07-01 03:00:00,2017-07-01 04:00:00,2017-07-01 05:00:00,2017-07-01 06:00:00,2017-07-01 07:00:00,2017-07-01 08:00:00,2017-07-01 09:00:00,...,2018-06-30 15:00:00,2018-06-30 16:00:00,2018-06-30 17:00:00,2018-06-30 18:00:00,2018-06-30 19:00:00,2018-06-30 20:00:00,2018-06-30 21:00:00,2018-06-30 22:00:00,2018-06-30 23:00:00,Cluster
X26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.179,1.703,1.504,1.271,1.255,1.596,1.379,1.000,0.976,TEST_Cluster1
X303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.211,0.168,0.140,0.495,0.175,1.692,0.883,0.539,0.481,TEST_Cluster2
X241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.245,0.211,0.215,0.372,0.430,0.332,0.435,0.268,0.220,TEST_Cluster2
X435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.208,0.207,0.199,0.331,0.429,0.366,0.497,0.459,0.418,TEST_Cluster2
X402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.188,0.179,0.213,0.232,0.169,0.159,0.166,0.463,0.646,TEST_Cluster2
X352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.757,0.523,0.530,0.625,0.739,0.255,0.233,0.454,0.419,TEST_Cluster2
X305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.344,0.316,0.513,0.561,0.573,0.501,0.634,0.676,0.680,TEST_Cluster2
X350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.345,0.324,0.322,0.398,0.341,0.281,0.401,0.543,0.499,TEST_Cluster2
X326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.272,0.571,0.148,0.134,0.127,0.126,0.439,0.737,0.304,TEST_Cluster2
X299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.110,0.301,0.223,0.347,0.286,0.230,0.346,0.466,0.405,TEST_Cluster2


In [123]:
# UDF of random sampling

def sampling_func(data, sample_pct):
    np.random.seed(123)
    N = len(data)
    sample_n = int(sample_pct) # integer
    sample = data.take(np.random.permutation(N)[:sample_n])
    return sample

In [124]:
len(train_cluster)

1300

In [129]:
train_group_set = train_cluster.groupby('Cluster').apply(sampling_func, 2)
train_group_set.sort_index()

Time                 2016-07-26 11:00:00  2016-07-26 12:00:00  \
Cluster                                                         
TRAIN_Cluster1 X11                   NaN                  NaN   
               X83                  1.41                1.549   
TRAIN_Cluster2 X429                  NaN                  NaN   
               X789                  NaN                  NaN   

Time                 2016-07-26 13:00:00  2016-07-26 14:00:00  \
Cluster                                                         
TRAIN_Cluster1 X11                   NaN                  NaN   
               X83                 3.834                4.845   
TRAIN_Cluster2 X429                  NaN                  NaN   
               X789                  NaN                  NaN   

Time                 2016-07-26 15:00:00  2016-07-26 16:00:00  \
Cluster                                                         
TRAIN_Cluster1 X11                   NaN                  NaN   
               X83                 4.336                1.909   
TRAIN_Cluster2 X429                  NaN                  NaN   
               X789                  NaN                  NaN   

Time                 2016-07-26 17:00:00  2016-07-26 18:00:00  \
Cluster                                                         
TRAIN_Cluster1 X11                   NaN                  NaN   
               X83                 1.926                2.894   
TRAIN_Cluster2 X429                  NaN                  NaN   
               X789                  NaN                  NaN   

Time                 2016-07-26 19:00:00  2016-07-26 20:00:00  ...  \
Cluster                                                        ...   
TRAIN_Cluster1 X11                   NaN                  NaN  ...   
               X83                 4.084                4.173  ...   
TRAIN_Cluster2 X429                  NaN                  NaN  ...   
               X789                  NaN                  NaN  ...   

Time                 2018-06-30 15:00:00  2018-06-30 16:00:00  \
Cluster                                                         
TRAIN_Cluster1 X11                 2.703                2.096   
               X83                 2.425                2.274   
TRAIN_Cluster2 X429                0.320                0.150   
               X789                0.411                0.413   

Time                 2018-06-30 17:00:00  2018-06-30 18:00:00  \
Cluster                                                         
TRAIN_Cluster1 X11                 2.081                2.128   
               X83                 3.182                2.748   
TRAIN_Cluster2 X429                0.116                0.110   
               X789                0.326                0.449   

Time                 2018-06-30 19:00:00  2018-06-30 20:00:00  \
Cluster                                                         
TRAIN_Cluster1 X11                 2.087                1.247   
               X83                 2.660                2.956   
TRAIN_Cluster2 X429                0.162                0.108   
               X789                0.526                0.590   

Time                 2018-06-30 21:00:00  2018-06-30 22:00:00  \
Cluster                                                         
TRAIN_Cluster1 X11                 1.161                0.990   
               X83                 2.832                2.795   
TRAIN_Cluster2 X429                0.105                0.228   
               X789                0.597                0.432   

Time                 2018-06-30 23:00:00         Cluster  
Cluster                                                   
TRAIN_Cluster1 X11                 1.018  TRAIN_Cluster1  
               X83                 3.370  TRAIN_Cluster1  
TRAIN_Cluster2 X429                0.212  TRAIN_Cluster2  
               X789                0.505  TRAIN_Cluster2  

[4 rows x 16910 columns]

In [137]:
test_group_set = test_cluster.groupby('Cluster').apply(sampling_func, 2)
test_group_set.sort_index()

Time                2017-07-01 00:00:00  2017-07-01 01:00:00  \
Cluster                                                        
TEST_Cluster1 X44                   NaN                  NaN   
              X55                   NaN                  NaN   
TEST_Cluster2 X406                  NaN                  NaN   
              X443                  NaN                  NaN   

Time                2017-07-01 02:00:00  2017-07-01 03:00:00  \
Cluster                                                        
TEST_Cluster1 X44                   NaN                  NaN   
              X55                   NaN                  NaN   
TEST_Cluster2 X406                  NaN                  NaN   
              X443                  NaN                  NaN   

Time                2017-07-01 04:00:00  2017-07-01 05:00:00  \
Cluster                                                        
TEST_Cluster1 X44                   NaN                  NaN   
              X55                   NaN                  NaN   
TEST_Cluster2 X406                  NaN                  NaN   
              X443                  NaN                  NaN   

Time                2017-07-01 06:00:00  2017-07-01 07:00:00  \
Cluster                                                        
TEST_Cluster1 X44                   NaN                  NaN   
              X55                   NaN                  NaN   
TEST_Cluster2 X406                  NaN                  NaN   
              X443                  NaN                  NaN   

Time                2017-07-01 08:00:00  2017-07-01 09:00:00  ...  \
Cluster                                                       ...   
TEST_Cluster1 X44                   NaN                  NaN  ...   
              X55                   NaN                  NaN  ...   
TEST_Cluster2 X406                  NaN                  NaN  ...   
              X443                  NaN                  NaN  ...   

Time                2018-06-30 15:00:00  2018-06-30 16:00:00  \
Cluster                                                        
TEST_Cluster1 X44                 0.552                0.299   
              X55                20.879               21.027   
TEST_Cluster2 X406                0.321                0.159   
              X443                0.378                0.459   

Time                2018-06-30 17:00:00  2018-06-30 18:00:00  \
Cluster                                                        
TEST_Cluster1 X44                 0.203                0.842   
              X55                21.257               20.465   
TEST_Cluster2 X406                0.194                0.247   
              X443                0.388                0.406   

Time                2018-06-30 19:00:00  2018-06-30 20:00:00  \
Cluster                                                        
TEST_Cluster1 X44                 6.624                0.028   
              X55                20.225               21.956   
TEST_Cluster2 X406                0.263                0.512   
              X443                0.428                0.413   

Time                2018-06-30 21:00:00  2018-06-30 22:00:00  \
Cluster                                                        
TEST_Cluster1 X44                 0.028                0.115   
              X55                19.645               19.543   
TEST_Cluster2 X406                0.473                0.530   
              X443                0.466                0.473   

Time                2018-06-30 23:00:00        Cluster  
Cluster                                                 
TEST_Cluster1 X44                 0.028  TEST_Cluster1  
              X55                19.220  TEST_Cluster1  
TEST_Cluster2 X406                0.411  TEST_Cluster2  
              X443                0.519  TEST_Cluster2  

[4 rows x 8761 columns]